# Cuestionamiento 3: Existe una relacion entre el exito ($ y ratings) de una pelicula y el director y/o budget?

In [1]:
# Import libraries
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats as st

In [2]:
# Import csv files
movies = pd.read_csv('IMDb movies.csv')
actors = pd.read_csv('IMDb names.csv')
ratings = pd.read_csv('IMDb ratings.csv')
title_principals = pd.read_csv('IMDb title_principals.csv')

In [90]:
movies = pd.DataFrame(movies)
actors = pd.DataFrame(actors)
ratings = pd.DataFrame(ratings)
title_principals = pd.DataFrame(title_principals)

In [9]:
movies.count()

imdb_title_id            81273
title                    81273
original_title           81273
year                     81273
date_published           81273
genre                    81273
duration                 81273
country                  81234
language                 80518
director                 81200
writer                   79780
production_company       76948
actors                   81207
description              78843
avg_vote                 81273
votes                    81273
budget                   22804
usa_gross_income         15094
worlwide_gross_income    29892
metascore                12722
reviews_from_users       74196
reviews_from_critics     70286
dtype: int64

In [6]:
ratings.count()

imdb_title_id                81273
weighted_average_vote        81273
total_votes                  81273
mean_vote                    81273
median_vote                  81273
votes_10                     81273
votes_9                      81273
votes_8                      81273
votes_7                      81273
votes_6                      81273
votes_5                      81273
votes_4                      81273
votes_3                      81273
votes_2                      81273
votes_1                      81273
allgenders_0age_avg_vote     26543
allgenders_0age_votes        26543
allgenders_18age_avg_vote    80858
allgenders_18age_votes       80858
allgenders_30age_avg_vote    81264
allgenders_30age_votes       81264
allgenders_45age_avg_vote    81160
allgenders_45age_votes       81160
males_allages_avg_vote       81272
males_allages_votes          81272
males_0age_avg_vote          20339
males_0age_votes             20339
males_18age_avg_vote         80217
males_18age_votes   

In [91]:
movies_ratings = pd.DataFrame.merge(movies,ratings[['imdb_title_id','us_voters_rating',
       'us_voters_votes', 'non_us_voters_rating', 'non_us_voters_votes']],how='left',on='imdb_title_id')
movies_ratings

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics,us_voters_rating,us_voters_votes,non_us_voters_rating,non_us_voters_votes
0,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,NaN,Charles Tait,...,$ 2250,NaN,NaN,NaN,7.0,7.0,6.0,89.0,6.2,309.0
1,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,NaN,NaN,NaN,NaN,4.0,2.0,6.2,27.0,6.0,114.0
2,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,...,$ 45000,NaN,NaN,NaN,24.0,3.0,5.5,197.0,4.7,103.0
3,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,NaN,NaN,NaN,NaN,28.0,14.0,7.1,452.0,7.0,1076.0
4,tt0002199,"From the Manger to the Cross; or, Jesus of Naz...","From the Manger to the Cross; or, Jesus of Naz...",1912,1913,"Biography, Drama",60,USA,English,Sidney Olcott,...,NaN,NaN,NaN,NaN,12.0,5.0,5.9,161.0,5.7,164.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81268,tt9903716,Jessie,Jessie,2019,2019-03-15,"Horror, Thriller",106,India,Telugu,Aswani Kumar V.,...,NaN,NaN,NaN,NaN,21.0,NaN,5.0,14.0,7.1,45.0
81269,tt9905412,Ottam,Ottam,2019,2019-03-08,Drama,120,India,Malayalam,Zam,...,INR 4000000,NaN,$ 4791,NaN,NaN,NaN,NaN,NaN,1.0,3.0
81270,tt9905462,Pengalila,Pengalila,2019,2019-03-08,Drama,111,India,Malayalam,T.V. Chandran,...,INR 10000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,3.0
81271,tt9911774,Padmavyuhathile Abhimanyu,Padmavyuhathile Abhimanyu,2019,2019-03-08,Drama,130,India,Malayalam,Vineesh Aaradya,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0


In [130]:
movies_ratings = movies_ratings.dropna(how='any')
movies_ratings2 =movies_ratings[~movies_ratings['budget'].str.match(r'([A-Z]+)')]
movies_ratings2 =movies_ratings2[~movies_ratings2['usa_gross_income'].str.match(r'($[A-Z]+\w)')]
movies_ratings2 =movies_ratings2[~movies_ratings2['worlwide_gross_income'].str.match(r'($[A-Z]+\w)')]
movies_ratings2['budget'] = movies_ratings2['budget'].str.replace('$','')
movies_ratings2['usa_gross_income'] = movies_ratings2['usa_gross_income'].str.replace('$','')
movies_ratings2['worlwide_gross_income'] = movies_ratings2['worlwide_gross_income'].str.replace('$','')
movies_ratings2

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics,us_voters_rating,us_voters_votes,non_us_voters_rating,non_us_voters_votes
1005,tt0021749,City Lights,City Lights,1931,1931-08-21,"Comedy, Drama, Romance",87,USA,English,Charles Chaplin,...,1500000,19181,32609,99.0,270.0,120.0,8.5,26265.0,8.5,69116.0
2336,tt0027977,Modern Times,Modern Times,1936,1936-10-16,"Comedy, Drama, Family",87,USA,English,Charles Chaplin,...,1500000,163577,445226,96.0,262.0,146.0,8.5,28002.0,8.5,92614.0
2687,tt0029583,Snow White and the Seven Dwarfs,Snow White and the Seven Dwarfs,1937,1938-07-08,"Animation, Family, Fantasy",83,USA,English,"William Cottrell, David Hand",...,1499000,184925486,184925486,95.0,229.0,166.0,7.7,37903.0,7.6,70466.0
3100,tt0031381,Gone with the Wind,Gone with the Wind,1939,1942-09-04,"Drama, History, Romance",238,USA,English,"Victor Fleming, George Cukor",...,3977000,200852579,402352579,97.0,808.0,183.0,8.2,63770.0,8.1,108032.0
3180,tt0031679,Mr. Smith Goes to Washington,Mr. Smith Goes to Washington,1939,1940-05-03,"Comedy, Drama",129,USA,English,Frank Capra,...,1900000,144738,144738,73.0,272.0,110.0,8.3,34275.0,8.0,32212.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80665,tt8772262,Midsommar,Midsommar,2019,2019-07-03,"Drama, Horror, Mystery",147,"USA, Sweden, Hungary","English, Swedish",Ari Aster,...,10000000,27426361,41123770,72.0,1454.0,365.0,7.5,12466.0,7.1,29604.0
80858,tt9024106,Unplanned,Unplanned,2019,2019-10-04,"Biography, Drama",109,USA,English,"Chuck Konzelman, Cary Solomon",...,6000000,19005109,19320481,10.0,531.0,32.0,6.1,2376.0,4.8,2524.0
80908,tt9082020,Cold Blood Legacy,Cold Blood Legacy,2019,2019-05-15,"Action, Thriller",91,"France, Ukraine, Belgium","French, English",Frédéric Petitjean,...,2700000,5083,1009888,25.0,72.0,26.0,4.5,214.0,4.5,1056.0
80946,tt9134216,Playing with Fire,Playing with Fire,2019,2019-11-08,"Comedy, Family",96,USA,English,Andy Fickman,...,29900000,18847824,21788746,24.0,49.0,16.0,4.7,336.0,4.3,190.0


In [131]:
movies_ratings2['budget'] = pd.to_numeric(movies_ratings2['budget'])
movies_ratings2['usa_gross_income'] = pd.to_numeric(movies_ratings2['usa_gross_income'])
movies_ratings2['worlwide_gross_income'] = pd.to_numeric(movies_ratings2['worlwide_gross_income'])


In [132]:
director = movies_ratings2.groupby('director')
director_movies = director['title'].value_counts()
director_totalbudget = director['budget'].sum()
director_meanbudget = director['budget'].mean()
director_usincome = director['usa_gross_income'].sum()
director_usincome = director['usa_gross_income'].mean()
director_usincome = director['worlwide_gross_income'].sum()
director_usincome = director['worlwide_gross_income'].mean()
director_totalbudget

director
Aaron Blaise, Robert Walker          128000000
Aaron Horvath, Peter Rida Michail     10000000
Aaron Schneider                        7000000
Aaron Seltzer, Jason Friedberg        20000000
Aaron Sorkin                          30000000
                                       ...    
Zara Hayes                            10000000
Zelda Barron                           5000000
Ziad Doueiri                           3000000
Zoe R. Cassavetes                      2000000
Élie Chouraqui                        32000000
Name: budget, Length: 2628, dtype: int64